In [10]:
from navec import Navec
import pandas as pd
import numpy as np
import pymorphy3
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kukof\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kukof\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')

In [3]:
def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...']
stop_words = stopwords.words("russian")
morph = pymorphy3.MorphAnalyzer()

In [4]:
df = pd.read_pickle('df.p', compression='gzip')
df.head()

,url,source,title,content,datetime,topic,year,month,weekday,len_title,len_content,lifetime
0,https://lenta.ru//news/2023/10/22/v-velikobrit...,lenta,В Великобритании слили в сеть личный номер тел...,Личный номер телефона премьер-министра Великоб...,2023-10-22,crime,2023,October,Sunday,68,674,27250.0
1,https://lenta.ru//news/2023/10/22/kitay-vyyavi...,lenta,Китай выявил шпионившего в пользу США на оборо...,Спецслужбы Китая выявили случай шпионажа в пол...,2023-10-22,crime,2023,October,Sunday,62,1063,27250.0
2,https://lenta.ru//news/2023/10/17/v-belgii-lik...,lenta,В Бельгии ликвидировали подозреваемого в напад...,"Полиция в Бельгии нейтрализовала человека, кот...",2023-10-17,crime,2023,October,Tuesday,61,842,34450.0
3,https://lenta.ru//news/2023/10/16/missiyu-oon-...,lenta,Миссию ООН обокрали в секторе Газа,Ближневосточное агентство ООН для помощи палес...,2023-10-16,crime,2023,October,Monday,34,1136,35890.0
4,https://lenta.ru//news/2023/10/13/vo-frantsii-...,lenta,Во Франции задержали еще одного человека с нож...,Французская полиция задержала еще одного воору...,2023-10-13,crime,2023,October,Friday,64,852,40210.0


In [5]:
df.topic.value_counts()

topic
politics     6204
culture      4343
sport        4190
science      3787
media        3655
incident     3472
society      2948
crime        2711
life         2296
accident     1674
russia       1087
economics    1068
army          900
social        346
travel        311
Name: count, dtype: int64

In [6]:
%%time
df['title'] = df.title
df['content'] = df.content
df['clean_title'] =df.title.apply(lambda x: preprocess(x, stop_words, punctuation_marks, morph))
df['clean_content'] =df.content.apply(lambda x: preprocess(x, stop_words, punctuation_marks, morph))

CPU times: total: 34min 19s
Wall time: 49min 4s


In [7]:
def natasha_embeddings(text, embedding_size = 300):

    text_embeddings = np.zeros(embedding_size)
    cnt = 0
    for tocken in text:
        embedding = navec.get(tocken, navec['<unk>'])
        text_embeddings += np.array(embedding)
        cnt += 1
    if cnt:
        text_embeddings = text_embeddings / cnt

    return text_embeddings

In [8]:
df['embeddings_title'] =df.clean_title.apply(lambda x: natasha_embeddings(x))
df['embeddings_content'] =df.clean_content.apply(lambda x: natasha_embeddings(x))

NameError: name 'y_train' is not defined

In [17]:
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.svm import SVC

In [12]:
X = np.hstack((np.array(list(df['embeddings_title'].values)), np.array(list(df['embeddings_content'].values))))
X_train, X_test, y_train, y_test = train_test_split(X, df.topic.values, random_state=42)
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [13]:
encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [18]:
models = [LogisticRegression(random_state=42, max_iter=10000), SVC(), CatBoostClassifier()]
for model_ in tqdm(models):
    print(model_)
    pipe = Pipeline([('clf', model_)])

    pipe.fit(list(X_train), y_train)
    y_pred = pipe.predict(list(X_test))
    print(classification_report(y_test, pipe.predict(list(X_test))))

  0%|          | 0/3 [00:00<?, ?it/s]

LogisticRegression(max_iter=10000, random_state=42)


 33%|███▎      | 1/3 [00:39<01:19, 39.97s/it]

              precision    recall  f1-score   support

           0       0.45      0.36      0.40       425
           1       0.55      0.45      0.49       237
           2       0.67      0.63      0.65       707
           3       0.52      0.62      0.57      1081
           4       0.43      0.29      0.35       263
           5       0.71      0.75      0.73       889
           6       0.30      0.18      0.22       571
           7       0.51      0.56      0.53       854
           8       0.63      0.76      0.69      1543
           9       0.50      0.31      0.38       286
          10       0.51      0.57      0.54       906
          11       0.28      0.06      0.10        83
          12       0.62      0.59      0.61       744
          13       0.57      0.57      0.57      1080
          14       0.00      0.00      0.00        79

    accuracy                           0.57      9748
   macro avg       0.48      0.45      0.46      9748
weighted avg       0.55   

C:\Users\kukof\PycharmProjects\pythonProject5\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kukof\PycharmProjects\pythonProject5\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kukof\PycharmProjects\pythonProject5\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

              precision    recall  f1-score   support

           0       0.50      0.36      0.42       425
           1       0.63      0.36      0.46       237
           2       0.69      0.66      0.68       707
           3       0.53      0.66      0.59      1081
           4       0.46      0.19      0.27       263
           5       0.70      0.76      0.73       889
           6       0.35      0.16      0.22       571
           7       0.52      0.63      0.57       854
           8       0.62      0.79      0.70      1543
           9       0.51      0.28      0.36       286
          10       0.55      0.65      0.60       906
          11       1.00      0.05      0.09        83
          12       0.69      0.53      0.60       744
          13       0.58      0.63      0.61      1080
          14       0.00      0.00      0.00        79

    accuracy                           0.59      9748
   macro avg       0.56      0.45      0.46      9748
weighted avg       0.58   

100%|██████████| 3/3 [52:38<00:00, 1052.84s/it]

              precision    recall  f1-score   support

           0       0.46      0.38      0.42       425
           1       0.73      0.53      0.61       237
           2       0.68      0.67      0.68       707
           3       0.53      0.61      0.57      1081
           4       0.37      0.27      0.31       263
           5       0.72      0.77      0.75       889
           6       0.28      0.16      0.20       571
           7       0.51      0.58      0.54       854
           8       0.65      0.77      0.70      1543
           9       0.44      0.34      0.38       286
          10       0.53      0.61      0.57       906
          11       0.52      0.14      0.23        83
          12       0.69      0.57      0.62       744
          13       0.57      0.58      0.57      1080
          14       0.00      0.00      0.00        79

    accuracy                           0.58      9748
   macro avg       0.51      0.47      0.48      9748
weighted avg       0.57   

In [19]:
models = [ RandomForestClassifier(), SGDClassifier(), OneVsRestClassifier(SVC()),]
for model_ in tqdm(models):
    print(model_)
    pipe = Pipeline([ ('clf', model_)])

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    print(classification_report(y_test, pipe.predict(X_test)))

  0%|          | 0/3 [00:00<?, ?it/s]

RandomForestClassifier()


 33%|███▎      | 1/3 [03:52<07:45, 232.70s/it]

              precision    recall  f1-score   support

           0       0.43      0.31      0.36       425
           1       0.80      0.14      0.23       237
           2       0.64      0.58      0.61       707
           3       0.43      0.49      0.46      1081
           4       0.14      0.06      0.08       263
           5       0.66      0.75      0.70       889
           6       0.19      0.13      0.16       571
           7       0.41      0.46      0.43       854
           8       0.59      0.81      0.68      1543
           9       0.30      0.15      0.20       286
          10       0.43      0.50      0.46       906
          11       0.33      0.01      0.02        83
          12       0.66      0.47      0.55       744
          13       0.48      0.51      0.50      1080
          14       0.00      0.00      0.00        79

    accuracy                           0.50      9748
   macro avg       0.43      0.36      0.36      9748
weighted avg       0.49   

 67%|██████▋   | 2/3 [04:01<01:41, 101.25s/it]

              precision    recall  f1-score   support

           0       0.34      0.37      0.35       425
           1       0.54      0.35      0.43       237
           2       0.65      0.67      0.66       707
           3       0.52      0.57      0.55      1081
           4       0.37      0.27      0.31       263
           5       0.67      0.77      0.72       889
           6       0.23      0.10      0.14       571
           7       0.45      0.65      0.53       854
           8       0.62      0.77      0.68      1543
           9       0.37      0.34      0.36       286
          10       0.59      0.46      0.51       906
          11       0.16      0.13      0.15        83
          12       0.63      0.55      0.59       744
          13       0.65      0.54      0.59      1080
          14       0.08      0.04      0.05        79

    accuracy                           0.55      9748
   macro avg       0.46      0.44      0.44      9748
weighted avg       0.54   

100%|██████████| 3/3 [1:11:24<00:00, 1428.18s/it]

              precision    recall  f1-score   support

           0       0.48      0.40      0.44       425
           1       0.66      0.32      0.43       237
           2       0.65      0.69      0.67       707
           3       0.56      0.58      0.57      1081
           4       0.44      0.17      0.25       263
           5       0.67      0.80      0.73       889
           6       0.29      0.11      0.16       571
           7       0.53      0.65      0.58       854
           8       0.63      0.79      0.70      1543
           9       0.45      0.31      0.37       286
          10       0.55      0.64      0.59       906
          11       0.24      0.07      0.11        83
          12       0.70      0.58      0.63       744
          13       0.60      0.64      0.62      1080
          14       0.00      0.00      0.00        79

    accuracy                           0.59      9748
   macro avg       0.50      0.45      0.46      9748
weighted avg       0.57   

In [20]:
#2 collumns
models = [LogisticRegression(random_state=42, max_iter=10000), SVC(), CatBoostClassifier()]
for model_ in tqdm(models):
    print(model_)
    pipe = Pipeline([('clf', model_)])

    pipe.fit(list(X_train), y_train)
    y_pred = pipe.predict(list(X_test))
    print(classification_report(y_test, pipe.predict(list(X_test))))

  0%|          | 0/6 [00:00<?, ?it/s]

LogisticRegression(max_iter=10000, random_state=42)


 17%|█▋        | 1/6 [00:39<03:19, 39.87s/it]

              precision    recall  f1-score   support

           0       0.45      0.36      0.40       425
           1       0.55      0.45      0.49       237
           2       0.67      0.63      0.65       707
           3       0.52      0.62      0.57      1081
           4       0.43      0.29      0.35       263
           5       0.71      0.75      0.73       889
           6       0.30      0.18      0.22       571
           7       0.51      0.56      0.53       854
           8       0.63      0.76      0.69      1543
           9       0.50      0.31      0.38       286
          10       0.51      0.57      0.54       906
          11       0.28      0.06      0.10        83
          12       0.62      0.59      0.61       744
          13       0.57      0.57      0.57      1080
          14       0.00      0.00      0.00        79

    accuracy                           0.57      9748
   macro avg       0.48      0.45      0.46      9748
weighted avg       0.55   

C:\Users\kukof\PycharmProjects\pythonProject5\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kukof\PycharmProjects\pythonProject5\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kukof\PycharmProjects\pythonProject5\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

              precision    recall  f1-score   support

           0       0.50      0.36      0.42       425
           1       0.63      0.36      0.46       237
           2       0.69      0.66      0.68       707
           3       0.53      0.66      0.59      1081
           4       0.46      0.19      0.27       263
           5       0.70      0.76      0.73       889
           6       0.35      0.16      0.22       571
           7       0.52      0.63      0.57       854
           8       0.62      0.79      0.70      1543
           9       0.51      0.28      0.36       286
          10       0.55      0.65      0.60       906
          11       1.00      0.05      0.09        83
          12       0.69      0.53      0.60       744
          13       0.58      0.63      0.61      1080
          14       0.00      0.00      0.00        79

    accuracy                           0.59      9748
   macro avg       0.56      0.45      0.46      9748
weighted avg       0.58   

 50%|█████     | 3/6 [1:01:50<1:01:50, 1236.96s/it]

              precision    recall  f1-score   support

           0       0.46      0.38      0.42       425
           1       0.73      0.53      0.61       237
           2       0.68      0.67      0.68       707
           3       0.53      0.61      0.57      1081
           4       0.37      0.27      0.31       263
           5       0.72      0.77      0.75       889
           6       0.28      0.16      0.20       571
           7       0.51      0.58      0.54       854
           8       0.65      0.77      0.70      1543
           9       0.44      0.34      0.38       286
          10       0.53      0.61      0.57       906
          11       0.52      0.14      0.23        83
          12       0.69      0.57      0.62       744
          13       0.57      0.58      0.57      1080
          14       0.00      0.00      0.00        79

    accuracy                           0.58      9748
   macro avg       0.51      0.47      0.48      9748
weighted avg       0.57   

AttributeError: 'list' object has no attribute '_validate_params'

 Корректируем список топиков. Смотрим качество.

In [21]:
dict = {'incident': 'crime', 'social': 'society'}
df.topic = df.topic.replace(dict)
df.topic.value_counts()

topic
politics     6204
crime        6183
culture      4343
sport        4190
science      3787
media        3655
society      3294
life         2296
accident     1674
russia       1087
economics    1068
army          900
travel        311
Name: count, dtype: int64

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, df.topic.values, random_state=42)
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

Корректируем добавляем новые стопы. Смотрим качество.

In [23]:
stop_words = stopwords.words("russian")
stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'за', 'из',
                   'из-за','на', 'ок', 'кстати', 'который', 'мочь', 'весь', 'еще', 'также', 'свой',
                   'ещё', 'самый', 'ул', 'комментарий', "известно", "оценил", "назвал", "нашли", "обнаружили", "оценили",
                   'рассказал', "рассказали",  "новости", "рассказала","названы", "назвали", "назвала", "заявил", "почему"])

In [24]:
df['clean_title'] =df.title.apply(lambda x: preprocess(x, stop_words, punctuation_marks, morph))
df['clean_content'] =df.content.apply(lambda x: preprocess(x, stop_words, punctuation_marks, morph))
df['embeddings_title'] =df.clean_title.apply(lambda x: natasha_embeddings(x))
df['embeddings_content'] =df.clean_content.apply(lambda x: natasha_embeddings(x))

In [25]:
X = np.hstack((np.array(list(df['embeddings_title'].values)), np.array(list(df['embeddings_content'].values))))
X_train, X_test, y_train, y_test = train_test_split(X, df.topic.values, random_state=42)
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [26]:
models = [LogisticRegression(random_state=42, max_iter=10000), SVC(), CatBoostClassifier()]
for model_ in tqdm(models):
    print(model_)
    pipe = Pipeline([('clf', model_)])

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    print(classification_report(y_test, pipe.predict(X_test)))

  0%|          | 0/5 [00:00<?, ?it/s]

LogisticRegression(max_iter=10000, random_state=42)


 20%|██        | 1/5 [00:27<01:49, 27.47s/it]

              precision    recall  f1-score   support

           0       0.45      0.34      0.39       425
           1       0.55      0.43      0.49       237
           2       0.66      0.72      0.69      1596
           3       0.53      0.61      0.57      1081
           4       0.42      0.29      0.35       263
           5       0.28      0.16      0.20       571
           6       0.51      0.56      0.54       854
           7       0.65      0.75      0.69      1543
           8       0.48      0.30      0.37       286
           9       0.52      0.57      0.54       906
          10       0.63      0.57      0.60       827
          11       0.58      0.58      0.58      1080
          12       0.00      0.00      0.00        79

    accuracy                           0.57      9748
   macro avg       0.48      0.45      0.46      9748
weighted avg       0.55      0.57      0.56      9748

SVC()


C:\Users\kukof\PycharmProjects\pythonProject5\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kukof\PycharmProjects\pythonProject5\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kukof\PycharmProjects\pythonProject5\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

              precision    recall  f1-score   support

           0       0.51      0.36      0.42       425
           1       0.62      0.36      0.45       237
           2       0.68      0.77      0.72      1596
           3       0.54      0.65      0.59      1081
           4       0.43      0.18      0.25       263
           5       0.38      0.15      0.21       571
           6       0.52      0.63      0.57       854
           7       0.64      0.77      0.70      1543
           8       0.52      0.30      0.38       286
           9       0.56      0.65      0.60       906
          10       0.69      0.51      0.59       827
          11       0.59      0.63      0.61      1080
          12       0.00      0.00      0.00        79

    accuracy                           0.59      9748
   macro avg       0.51      0.46      0.47      9748
weighted avg       0.58      0.59      0.58      9748

Learning rate set to 0.093992
0:	learn: 2.4332304	total: 1.98s	remaining: 32m 5

 60%|██████    | 3/5 [50:06<33:24, 1002.03s/it]

              precision    recall  f1-score   support

           0       0.48      0.39      0.43       425
           1       0.70      0.54      0.61       237
           2       0.69      0.77      0.73      1596
           3       0.54      0.61      0.57      1081
           4       0.33      0.23      0.27       263
           5       0.29      0.15      0.19       571
           6       0.52      0.58      0.55       854
           7       0.67      0.75      0.71      1543
           8       0.43      0.32      0.37       286
           9       0.53      0.62      0.57       906
          10       0.70      0.55      0.61       827
          11       0.56      0.57      0.57      1080
          12       0.00      0.00      0.00        79

    accuracy                           0.58      9748
   macro avg       0.49      0.47      0.47      9748
weighted avg       0.57      0.58      0.57      9748

<class 'sklearn.ensemble._forest.RandomForestClassifier'>


AttributeError: 'numpy.ndarray' object has no attribute '_validate_params'

#! TfidfEmbeddingVectorizer

In [27]:
from collections import defaultdict
class TfidfEmbeddingVectorizer(object):
    """Get tfidf weighted vectors"""
    def __init__(self, model):
        self.model = navec
        self.dim = 300

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of
        # known idf's
        max_idf = max(tfidf.idf_)
        self.model = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.navec.get_vector(w) * self.navec[w]
                         for w in words if w in self.model_] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [28]:
stop_words = stopwords.words("russian")
stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'за', 'из',
                   'из-за','на', 'ок', 'кстати', 'который', 'мочь', 'весь', 'еще', 'также', 'свой',
                   'ещё', 'самый', 'ул', 'комментарий', "известно", "оценил", "назвал", "нашли", "обнаружили", "оценили",
                   'рассказал', "рассказали",  "новости", "рассказала","названы", "назвали", "назвала", "заявил", "почему"])
tfv = TfidfVectorizer(min_df=3,  max_features=None,
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = stop_words)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(df.clean_content.values, df.topic.values, random_state=42)
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [30]:
X_train

array([list(['глава', 'мид', 'израиль', 'эли', 'коэн', 'заявить', 'большой', 'видеться', 'генеральный', 'секретарь', 'оон', 'антониу', 'гутерриш', 'израильский', 'дипломат', 'написать', 'х', 'бывший', 'twitter', '«', 'встречаться', 'генеральный', 'секретарь', 'оон', '7', 'октябрь', 'остаться', 'место', 'сбалансированный', 'подход', 'хамас', 'должный', 'стёртый', 'лицо', 'земля', '»', 'написать', 'глава', 'израильский', 'мид.помить', 'постоянный', 'представитель', 'израиль', 'оон', 'гилада', 'эрдать', 'призвать', 'гутерриш', 'немедленно', 'уйти', 'отставку.недовольство', 'сторона', 'тель-авив', 'вызвать', 'слово', 'гутерриш', 'первопричина', 'атака', 'хамас', 'еврейский', 'государство', 'гутерриш', 'заявить', 'осуждать', 'атака', 'боевик', 'палестинский', 'исламистский', 'движение', 'хамас', 'израиль', 'напомнить', 'произойти', 'неспроста']),
       list(['сингапур', 'врач', 'извлечь', 'целое', 'осьминог', 'пищевод', 'пациент', 'жаловаться', 'тошнота', 'рвота', 'сообщать', 'daily', 'mai

In [31]:
tfidf = TfidfVectorizer(stop_words=stop_words)
X_train = tfidf.fit_transform(pd.Series(X_train).apply(lambda x: ' '.join(x)))

In [32]:
X_test = tfidf.transform(pd.Series(X_test).apply(lambda x: ' '.join(x)))

In [33]:

# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
svd = decomposition.TruncatedSVD(n_components=180)
svd.fit(X_train)
xtrain_svd = svd.transform(X_train)
xtest_svd = svd.transform(X_test)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xtest_svd_scl = scl.transform(xtest_svd)

In [34]:
models = [LogisticRegression(random_state=42, max_iter=10000), SVC(), CatBoostClassifier()]
for model_ in tqdm(models):
    print(model_)
    pipe = Pipeline([('clf', model_)])

    pipe.fit(xtrain_svd_scl, y_train)
    y_pred = pipe.predict(xtest_svd_scl)
    print(classification_report(y_test, y_pred))

  0%|          | 0/3 [00:00<?, ?it/s]

LogisticRegression(max_iter=10000, random_state=42)


 33%|███▎      | 1/3 [00:11<00:23, 11.52s/it]

              precision    recall  f1-score   support

           0       0.46      0.37      0.41       425
           1       0.73      0.68      0.70       237
           2       0.69      0.73      0.71      1596
           3       0.57      0.63      0.60      1081
           4       0.49      0.32      0.39       263
           5       0.35      0.24      0.28       571
           6       0.54      0.61      0.58       854
           7       0.68      0.73      0.71      1543
           8       0.52      0.41      0.46       286
           9       0.55      0.60      0.57       906
          10       0.64      0.60      0.62       827
          11       0.61      0.62      0.61      1080
          12       0.14      0.05      0.07        79

    accuracy                           0.60      9748
   macro avg       0.54      0.51      0.52      9748
weighted avg       0.59      0.60      0.60      9748

SVC()


C:\Users\kukof\PycharmProjects\pythonProject5\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kukof\PycharmProjects\pythonProject5\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kukof\PycharmProjects\pythonProject5\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

              precision    recall  f1-score   support

           0       0.54      0.49      0.51       425
           1       0.76      0.62      0.68       237
           2       0.72      0.78      0.75      1596
           3       0.62      0.67      0.64      1081
           4       0.65      0.28      0.39       263
           5       0.44      0.22      0.29       571
           6       0.61      0.71      0.66       854
           7       0.66      0.76      0.71      1543
           8       0.53      0.44      0.48       286
           9       0.61      0.69      0.65       906
          10       0.72      0.62      0.66       827
          11       0.66      0.69      0.67      1080
          12       0.00      0.00      0.00        79

    accuracy                           0.65      9748
   macro avg       0.58      0.54      0.55      9748
weighted avg       0.64      0.65      0.64      9748

Learning rate set to 0.093992
0:	learn: 2.3941655	total: 636ms	remaining: 10m 3

100%|██████████| 3/3 [18:49<00:00, 376.62s/it]

              precision    recall  f1-score   support

           0       0.51      0.44      0.47       425
           1       0.73      0.59      0.65       237
           2       0.71      0.75      0.73      1596
           3       0.57      0.62      0.60      1081
           4       0.45      0.29      0.35       263
           5       0.34      0.18      0.24       571
           6       0.58      0.64      0.61       854
           7       0.67      0.76      0.71      1543
           8       0.47      0.44      0.46       286
           9       0.57      0.65      0.61       906
          10       0.71      0.62      0.66       827
          11       0.61      0.61      0.61      1080
          12       0.00      0.00      0.00        79

    accuracy                           0.61      9748
   macro avg       0.53      0.51      0.51      9748
weighted avg       0.60      0.61      0.61      9748



# Glove

In [35]:
import gensim.downloader
from gensim.models import Word2Vec, KeyedVectors

In [36]:
glove = gensim.downloader.load('glove-twitter-200')

In [37]:
class TfidfEmbeddingVectorizer(object):
    """Get tfidf weighted vectors"""
    def __init__(self, model):
        self.word2vec = model
        self.word2weight = None
        self.dim = model.vector_size

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec.get_vector(w) * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [ ]:
models = [LogisticRegression(random_state=42, max_iter=10000), SVC(), CatBoostClassifier()]
for model_ in tqdm(models):
    print(model_)
    pipe = Pipeline([('w2v', TfidfEmbeddingVectorizer(glove)),
                     ('clf', model_)])

    pipe.fit(xtrain_svd_scl, y_train)
    y_pred = pipe.predict(xtest_svd_scl)
    print(classification_report(y_test, y_pred))

  0%|          | 0/3 [00:00<?, ?it/s]

LogisticRegression(max_iter=10000, random_state=42)


In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
models = [ RandomForestClassifier(), SGDClassifier(), OneVsRestClassifier(SVC()),]
for model_ in tqdm(models):
    print(model_)
    pipe = Pipeline([('w2v', TfidfEmbeddingVectorizer(glove)),
                     ('clf', model_)])

    pipe.fit(xtrain_svd_scl, y_train)
    y_pred = pipe.predict(xtest_svd_scl)
    print(classification_report(y_test, y_pred))